**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
%pip install scikit-video

     |████████████████████████████████| 2.3MB 4.8MB/s 


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function `act` returns the action we will chose. We are performing $\epsilon$-greedy policy:

We take a random probability $p$ (which is equivalent to taking a random float following a uniform law on $[0, 1]$. If $p > \epsilon$, we take the action by learning. We take a random action otherwise.


$\epsilon$ is important because it enables a trade-off between exploration and exploitation. This is a famous dilemma in Reinforcement learning. We want to explore as much as possible but, if we do it all the time, we will be penalized because we will do action with bad reward. Exploitation could enable us to take the actions with high reward because we know it. If we are exploiting too much, we can miss action with high reward. Setting $\epsilon$ properly is important to have a good trade off in order to maximize the value function.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1 #Deux dernieres colonnes non?

        self.position[self.x, self.y] = 1 #The cell is visited
        #Actions: up, down, left, right
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1 #if can't, go left
            else:
                self.x = self.x + 1 #go right
        elif action == 1: 
            if self.x == 2:
                self.x = self.x+1 #if can't, go right
            else:
                self.x = self.x-1 #go left
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1 #if can't, go up
            else:
                self.y = self.y + 1 #go down
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1 #if can't go down
            else:
                self.y = self.y - 1 #go up
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y] #board contains the values and the location of cheese and poison
        self.board[self.x, self.y] = 0 #Already visited, cheese or poisen eaten
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:] #Visible grid for the rat

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=40 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.



*   ```position``` is an array to remember where the rat has already been. We are puting a $1$ in all the cells that are visited by the rat in its journey. The default value is $0$. We are puting a $-1$ values for all the cell the rat is not supposed to visit to explore: it has a visibility of $2$ so it doesn't need to go in the first two rows and columns and the two last ones to know if there is cheere or poison in theses cells. 
*   ```board``` is an array which contains the location of the cheese and the poison. These locations are in fact floats corresponding to the reward (different reward values for the cheese or the poison.)



## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #Just chose action randomly
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:

    

def test(agent,env,epochs,prefix=''):

    
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### 
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action)

            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward
        #####
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.5/8.0. Average score (-0.5)
Win/lose count 9.5/8.0. Average score (0.5)
Win/lose count 7.5/11.0. Average score (-0.8333333333333334)
Win/lose count 9.5/11.0. Average score (-1.0)
Win/lose count 9.5/15.0. Average score (-1.9)
Win/lose count 7.5/16.0. Average score (-3.0)
Win/lose count 8.0/18.0. Average score (-4.0)
Win/lose count 11.0/14.0. Average score (-3.875)
Win/lose count 5.0/9.0. Average score (-3.888888888888889)
Win/lose count 9.0/10.0. Average score (-3.6)
Win/lose count 8.5/19.0. Average score (-4.2272727272727275)
Win/lose count 10.5/10.0. Average score (-3.8333333333333335)
Win/lose count 8.0/11.0. Average score (-3.769230769230769)
Win/lose count 11.0/19.0. Average score (-4.071428571428571)
Win/lose count 11.5/15.0. Average score (-4.033333333333333)
Win/lose count 8.0/8.0. Average score (-3.78125)
Win/lose count 10.5/11.0. Average score (-3.588235294117647)
Win/lose count 14.0/15.0. Average score (-3.4444444444444446)
Win/lose count 13.0/15.0. Average s

The final score isn't positive at all! The rat isn't winning. There is no learning because we chose a random agent.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}








*    First part of the answer

\begin{align*}
    Q^{\pi}(s,a) &= E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
                &= E[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
                &= r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] \\
                &= r(s,a) + \gamma \sum_{s',a'}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
                &= \sum_{s',a'} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \gamma \sum_{s',a'}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
                &= E_{s',a'\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

*    Second part

We want to compute the optimum $Q^* = Q^{\pi^*}= \max_{\pi} Q^{\pi}(s,a)$

As $\pi^*$ is a optimal policy, we know that: 
$$Q^{\pi^*}(s) = \max_{a'} Q^{\pi^*}(s, a')$$

\begin{align*}
Q^{\pi*}(s,a) &= E_{s',a'\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi*}(s',a')] \\
            &= E_{s',a'\sim p(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{\pi^*}(s', a')] \\
            &= E_{s',a'\sim p(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s', a')] \\ 
\end{align*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        #remember m
        self.memory.append(m)
        #Adjust the size of the buffer
        if len(self.memory) > self.max_memory:
          #Remove the first one to adjust the length
          del self.memory[0]

    def random_access(self):
        #Pick random int i and return memory[i]
        i = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[i] #state, n_state, action, reward, game_over

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, state):
        #reshape state
        state_re = state.reshape([1, state.shape[0], state.shape[1], state.shape[2]])
        #Action maximizing the predicted target
        return np.argmax(self.model.predict(state_re)[0, :])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            #Random access of the memorized displacement
            state, n_state, action, reward, game_over = self.memory.random_access()

            #State is a tensor, reshape it
            state_re = state.reshape([1, state.shape[0], state.shape[1], state.shape[2]])
            #Compute the target thanks to the model
            target_q[i] = self.model.predict(state_re)[0] 
            
            if game_over_:
                ######## FILL IN
                #We don't have any new information, no improvement
                input_states[i] = state
                target_q[i, action] = reward
            else:
                ######## FILL IN
                #Improvement
                input_states[i] = state
                #For each state, compute the q values
                n_state_re = n_state.reshape([1, n_state.shape[0], n_state.shape[1], n_state.shape[2]])
                #Compute Q*
                target_q[i, action] = reward + self.discount * max(self.model.predict(n_state_re)[0])

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(4)) #Dimension of the target_q
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [15]:
# Set parameters values
size = 13
T=200
temperature=0.3
epochs_train=40 # set small when debugging
epochs_test=20 # set small when debugging


env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
#HTML(display_videos('fc_train9.mp4'))

Epoch 000/040 | Loss 0.1640 | Win/lose count 6.5/4.0 (2.5)
Epoch 001/040 | Loss 1.1351 | Win/lose count 6.5/3.0 (3.5)
Epoch 002/040 | Loss 1.6222 | Win/lose count 4.0/1.0 (3.0)
Epoch 003/040 | Loss 1.7030 | Win/lose count 4.0/2.0 (2.0)
Epoch 004/040 | Loss 1.8792 | Win/lose count 6.5/5.0 (1.5)
Epoch 005/040 | Loss 1.7856 | Win/lose count 4.0/5.0 (-1.0)
Epoch 006/040 | Loss 1.6046 | Win/lose count 5.5/5.0 (0.5)
Epoch 007/040 | Loss 1.7033 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/040 | Loss 1.6915 | Win/lose count 4.0/4.0 (0.0)
Epoch 009/040 | Loss 1.7387 | Win/lose count 2.5/3.0 (-0.5)
Epoch 010/040 | Loss 1.6950 | Win/lose count 3.5/7.0 (-3.5)
Epoch 011/040 | Loss 1.6437 | Win/lose count 4.0/2.0 (2.0)
Epoch 012/040 | Loss 1.7224 | Win/lose count 6.5/2.0 (4.5)
Epoch 013/040 | Loss 1.7137 | Win/lose count 1.0/0 (1.0)
Epoch 014/040 | Loss 1.6071 | Win/lose count 2.0/4.0 (-2.0)
Epoch 015/040 | Loss 1.4564 | Win/lose count 2.0/3.0 (-1.0)
Epoch 016/040 | Loss 1.1900 | Win/lose count 6.0/7.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(30, (2, 2), input_shape=(5, 5, self.n_state,), activation='relu'))
        model.add(Conv2D(30, (2, 2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
#HTML(display_videos('cnn_train9.mp4'))

Epoch 000/040 | Loss 0.0076 | Win/lose count 3.5/7.0 (-3.5)
Epoch 001/040 | Loss 0.0294 | Win/lose count 3.0/1.0 (2.0)
Epoch 002/040 | Loss 0.0605 | Win/lose count 1.5/2.0 (-0.5)
Epoch 003/040 | Loss 0.2787 | Win/lose count 4.5/4.0 (0.5)
Epoch 004/040 | Loss 1.1157 | Win/lose count 0.5/1.0 (-0.5)
Epoch 005/040 | Loss 1.9891 | Win/lose count 4.0/1.0 (3.0)
Epoch 006/040 | Loss 1.9015 | Win/lose count 4.5/2.0 (2.5)
Epoch 007/040 | Loss 2.0231 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/040 | Loss 1.9571 | Win/lose count 11.0/3.0 (8.0)
Epoch 009/040 | Loss 1.9486 | Win/lose count 6.5/4.0 (2.5)
Epoch 010/040 | Loss 1.8471 | Win/lose count 6.5/2.0 (4.5)
Epoch 011/040 | Loss 1.7962 | Win/lose count 5.5/3.0 (2.5)
Epoch 012/040 | Loss 1.9832 | Win/lose count 11.5/8.0 (3.5)
Epoch 013/040 | Loss 1.9105 | Win/lose count 7.0/6.0 (1.0)
Epoch 014/040 | Loss 1.8267 | Win/lose count 11.0/1.0 (10.0)
Epoch 015/040 | Loss 1.8561 | Win/lose count 6.5/4.0 (2.5)
Epoch 016/040 | Loss 1.8055 | Win/lose count 6.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
temperature_li = [0.1 * i for i in range(1, 10)]

In [19]:
for temperature in temperature_li:
    print('Temperature = ', temperature)
    env = Environment(grid_size=size, max_time=T,temperature=0.3)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')

Temperature =  0.1
Test of the CNN
Win/lose count 3.5/2.0. Average score (1.5)
Win/lose count 8.0/5.0. Average score (2.25)
Win/lose count 6.0/2.0. Average score (2.8333333333333335)
Win/lose count 13.5/1.0. Average score (5.25)
Win/lose count 9.5/3.0. Average score (5.5)
Win/lose count 3.0/2.0. Average score (4.75)
Win/lose count 5.5/2.0. Average score (4.571428571428571)
Win/lose count 1.5/0. Average score (4.1875)
Win/lose count 9.0/1.0. Average score (4.611111111111111)
Win/lose count 7.0/5.0. Average score (4.35)
Win/lose count 14.0/3.0. Average score (4.954545454545454)
Win/lose count 3.5/0. Average score (4.833333333333333)
Win/lose count 4.5/3.0. Average score (4.576923076923077)
Win/lose count 7.0/1.0. Average score (4.678571428571429)
Win/lose count 1.0/1.0. Average score (4.366666666666666)
Win/lose count 7.0/1.0. Average score (4.46875)
Win/lose count 10.5/7.0. Average score (4.411764705882353)
Win/lose count 7.0/2.0. Average score (4.444444444444445)
Win/lose count 7.5/3.0

In [0]:
#HTML(display_videos('cnn_test10.mp4'))

In [0]:
#HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, decay_factor=0.8, prefix=''):
    #Using decreasing epsilon greedy exploration to tackle the excessive exploration
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        #Decay for decreasing epsilon
        new_epsilon = agent.epsilon * decay_factor
        agent.set_epsilon(new_epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        ###########################################
        #Will help us penalized the visited state to decrease exploration
        self.malus_position = np.zeros((grid_size, grid_size))
        ###########################################

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1 #Deux dernieres colonnes non?

        self.position[self.x, self.y] = 1 #The cell is visited
        #Actions: up, down, left, right
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1 #if can't, go left
            else:
                self.x = self.x + 1 #go right
        elif action == 1: 
            if self.x == 2:
                self.x = self.x+1 #if can't, go right
            else:
                self.x = self.x-1 #go left
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1 #if can't, go up
            else:
                self.y = self.y + 1 #go down
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1 #if can't go down
            else:
                self.y = self.y - 1 #go up
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        
        #############################################
        #We want to penalize the agent when there is an exploration
        if train:
            reward = -self.malus_position[self.x, self.y] #Lowing reward if the sate has been visited as a penality
        #Declare the state visited
        self.malus_position[self.x, self.y] = 0.2
        #############################################

        #Regular reward
        reward += self.board[self.x, self.y] #board contains the values and the location of cheese and poison
        self.board[self.x, self.y] = 0 #Already visited, cheese or poisen eaten
        game_over = self.t > self.max_time

        #Add a feature for the state
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:] 

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        #We add a pseudo state to know if it was visited, add a feature for the state
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state






In [23]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
#HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/040 | Loss 0.4820 | Win/lose count 13.0/42.20000000000007 (-29.200000000000067)
Epoch 001/040 | Loss 1.3350 | Win/lose count 5.3/38.80000000000002 (-33.50000000000002)
Epoch 002/040 | Loss 1.3209 | Win/lose count 9.8/30.799999999999926 (-20.999999999999925)
Epoch 003/040 | Loss 1.4044 | Win/lose count 5.199999999999998/38.60000000000001 (-33.40000000000001)
Epoch 004/040 | Loss 1.4268 | Win/lose count 11.400000000000006/32.99999999999993 (-21.599999999999923)
Epoch 005/040 | Loss 1.4474 | Win/lose count 9.1/37.199999999999996 (-28.099999999999994)
Epoch 006/040 | Loss 1.4050 | Win/lose count 8.599999999999998/37.59999999999998 (-28.999999999999982)
Epoch 007/040 | Loss 1.5265 | Win/lose count 6.6999999999999975/35.99999999999997 (-29.299999999999976)
Epoch 008/040 | Loss 1.6002 | Win/lose count 3.1999999999999997/38.80000000000002 (-35.600000000000016)
Epoch 009/040 | Loss 1.6746 | Win/lose count 5.399999999999999/38.20000000000001 (-32.80000000000001)
Epoch 010/040 | Loss 1.

In [24]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
#HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 5.0/2.0. Average score (3.0)
Win/lose count 1.0/0. Average score (2.0)
Win/lose count 18.0/6.0. Average score (5.333333333333333)
Win/lose count 3.5/0. Average score (4.875)
Win/lose count 1.0/0. Average score (4.1)
Win/lose count 2.5/2.0. Average score (3.5)
Win/lose count 6.0/2.0. Average score (3.5714285714285716)
Win/lose count 8.0/2.0. Average score (3.875)
Win/lose count 1.0/0. Average score (3.5555555555555554)
Win/lose count 9.0/0. Average score (4.1)
Win/lose count 14.5/5.0. Average score (4.590909090909091)
Win/lose count 3.0/0. Average score (4.458333333333333)
Win/lose count 2.0/2.0. Average score (4.115384615384615)
Win/lose count 9.0/3.0. Average score (4.25)
Win/lose count 8.5/3.0. Average score (4.333333333333333)
Win/lose count 4.0/3.0. Average score (4.125)
Win/lose count 6.0/0. Average score (4.235294117647059)
Win/lose count 21.5/6.0. Average score (4.861111111111111)
Win/lose count 14.0/2.0. Average score (5.2368421052631575)
Win/lose count 1.5/0. Av

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
class DQN_winning(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_winning, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #reshape state
        state_re = state.reshape([1, state.shape[0], state.shape[1], state.shape[2]])
        #Action maximizing the predicted target
        return np.argmax(self.model.predict(state_re)[0, :])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            state, n_state, action, reward, game_over = self.memory.random_access()
            #Copy the expert!
            input_states[i] = state
            #target_q are supposed to be maximum because we are copying the agent, but no good values found
            target_q[i, action] = 10

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_CNN_winning(DQN_winning):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_CNN_winning, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Conv2D(30, (2, 2), input_shape=(5, 5, self.n_state,), activation='relu'))
        model.add(Conv2D(30, (2, 2), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
def train_winning(expert, agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The expert performs an action
            action = expert.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy for the agent( copying the expert)
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [38]:
# Training
expert = agent
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_copy = DQN_CNN_winning(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_winning(expert, agent_copy, env, epochs_train, prefix='cnn_train_explore')
#HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/040 | Loss 13.9710 | Win/lose count 6.5/3.0 (3.5)
Epoch 001/040 | Loss 15.3261 | Win/lose count 2.5/1.0 (1.5)
Epoch 002/040 | Loss 13.1768 | Win/lose count 7.5/6.0 (1.5)
Epoch 003/040 | Loss 13.6041 | Win/lose count 5.0/1.0 (4.0)
Epoch 004/040 | Loss 14.6591 | Win/lose count 8.0/0 (8.0)
Epoch 005/040 | Loss 14.1360 | Win/lose count 2.5/0 (2.5)
Epoch 006/040 | Loss 13.6865 | Win/lose count 13.0/2.0 (11.0)
Epoch 007/040 | Loss 16.1571 | Win/lose count 16.5/0 (16.5)
Epoch 008/040 | Loss 13.6433 | Win/lose count 7.0/1.0 (6.0)
Epoch 009/040 | Loss 13.0540 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/040 | Loss 14.1684 | Win/lose count 1.5/0 (1.5)
Epoch 011/040 | Loss 14.6622 | Win/lose count 13.5/7.0 (6.5)
Epoch 012/040 | Loss 14.8797 | Win/lose count 5.5/0 (5.5)
Epoch 013/040 | Loss 13.4231 | Win/lose count 8.0/0 (8.0)
Epoch 014/040 | Loss 12.9807 | Win/lose count 1.0/0 (1.0)
Epoch 015/040 | Loss 12.3790 | Win/lose count 11.5/0 (11.5)
Epoch 016/040 | Loss 14.0003 | Win/lose count 2.5

I think this method wasn't really efficient. I just tried to copy the expert but the results are not satisfying.

***